<a href="https://colab.research.google.com/github/Apekshaa2908/Dynamic-Data-Ingestion-and-Storage-in-HDFS-with-Automated-Hive-Integration/blob/main/Dynamic_Data_Ingestion_and_Storage_in_HDFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Download and Install Hadoop and Hive



In [ ]:
#Hive installation
wget https://downloads.apache.org/hive/hive-4.0.0/apache-hive-4.0.0-bin.tar.gz

#Hadoop installation
wget https://downloads.apache.org/hadoop/common/hadoop-3.4.0/hadoop-3.4.0.tar.gz




 Download the Data File

In [ ]:
# Download the Census data CSV file
wget https://www2.census.gov/programs-surveys/popest/datasets/2020-2023/cities/totals/sub-est2023_32.csv


 HDFS Setup

In [ ]:
# List contents of /user directory in HDFS
hadoop fs -ls /user

# Create necessary directories in HDFS
hadoop fs -mkdir /user/hadoop
hadoop fs -mkdir /user/hadoop/my_directory

# Upload the CSV file to HDFS
hadoop fs -put sub-est2023_32.csv /user/hadoop/my_directory/


 Hive Setup and Data Loading

In [ ]:
# Start Hive CLI
hive

# Show existing databases
SHOW DATABASES;

# Create a new database
CREATE DATABASE myproject;

# Use the newly created database
USE myproject;

# Drop the table if it already exists
DROP TABLE IF EXISTS my_table;

# Create the table with the specified schema
CREATE TABLE my_table (
    SUMLEV INT,
    STATE INT,
    COUNTY INT,
    PLACE INT,
    COUSUB INT,
    CONCIT INT,
    PRIMGEO_FLAG INT,
    FUNCSTAT STRING,
    NAME STRING,
    STNAME STRING,
    ESTIMATESBASE2020 INT,
    POPESTIMATE2020 INT,
    POPESTIMATE2021 INT,
    POPESTIMATE2022 INT,
    POPESTIMATE2023 INT,
    PRIMARY KEY (STATE, COUNTY, PLACE, NAME) DISABLE NOVALIDATE
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE;

# Load data into the Hive table from HDFS
LOAD DATA INPATH '/user/hadoop/my_directory/sub-est2023_32.csv' INTO TABLE my_table;

# Query the table to verify data load
SELECT * FROM my_table LIMIT 10;


In [ ]:
nano hive_automation.sh

In [ ]:
#!/bin/bash

# Define variables
HDFS_DIR="/user/hadoop/my_directory"
LOCAL_FILE="sub-est2023_32.csv"
HDFS_PATH="$HDFS_DIR/$LOCAL_FILE"
HIVE_DB="myproject"
HIVE_TABLE="my_table"
SOURCE_URL="https://www2.census.gov/programs-surveys/popest/datasets/2020-2023/cities/totals/$LOCAL_FILE"

# 1. Download the CSV file using wget
wget $SOURCE_URL -O $LOCAL_FILE

# 2. Check if the HDFS directory exists, create if not
hadoop fs -test -d $HDFS_DIR
if [ $? -ne 0 ]; then
  echo "Creating HDFS directory: $HDFS_DIR"
  hadoop fs -mkdir -p $HDFS_DIR
fi

# 3. Upload the CSV file to HDFS
echo "Uploading $LOCAL_FILE to HDFS: $HDFS_PATH"
hadoop fs -put -f $LOCAL_FILE $HDFS_PATH

# 4. Start Hive CLI and execute the Hive commands
echo "Starting Hive process"
hive -e "
  -- Create database if it does not exist
  CREATE DATABASE IF NOT EXISTS $HIVE_DB;

  -- Use the database
  USE $HIVE_DB;

  -- Drop the table if it already exists
  DROP TABLE IF EXISTS $HIVE_TABLE;

  -- Create the table with schema matching the CSV file
  CREATE TABLE $HIVE_TABLE (
      SUMLEV INT,
      STATE INT,
      COUNTY INT,
      PLACE INT,
      COUSUB INT,
      CONCIT INT,
      PRIMGEO_FLAG INT,
      FUNCSTAT STRING,
      NAME STRING,
      STNAME STRING,
      ESTIMATESBASE2020 INT,
      POPESTIMATE2020 INT,
      POPESTIMATE2021 INT,
      POPESTIMATE2022 INT,
      POPESTIMATE2023 INT
  )
  ROW FORMAT DELIMITED
  FIELDS TERMINATED BY ','
  STORED AS TEXTFILE;

  -- Load data into the Hive table from HDFS
  LOAD DATA INPATH '$HDFS_PATH' INTO TABLE $HIVE_TABLE;

  -- Query the table to verify data load
  SELECT * FROM $HIVE_TABLE LIMIT 10;
"

# 5. Clean up local file if needed
echo "Cleaning up local CSV file"
rm $LOCAL_FILE

echo "Automation script completed successfully."


In [ ]:
chmod +x hive_automation.sh
./hive_automation.sh